# Higher-order functions

Higher-order functions (HOFs) are functions which receive or return other functions. HOFs show that functions can be regarded as _values_, much in the same way than integers, booleans, product values, sum values, etc. We will see that higher-order functions are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

## Functions as values

As we saw, functions are represented through methods in Scala (as in any other object-oriented language). But methods themselves can't be passed around and be returned by invocations. Therefore, the first thing to do in order to create HOFs in Scala is finding a way to reify methods. For instance, let's consider the following functions:

In [1]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

We want to implement a function that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and call this function over a given number. We may want to write something like this:

In [1]:
// def call(int2int(number: Int): Int, number: Int): Int = ???

where the first argument `int2int` attempt to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

In [2]:
abstract class FunctionInt2Int{
    def apply(number: Int): Int
}

defined class FunctionInt2Int

Now, we can implement the `call` HOF as follows: 

In [3]:
def call(int2int: FunctionInt2Int, number: Int): Int = 
    int2int.apply(number)

defined function call

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [4]:
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}

val substractOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number - 1 
}

addOneV: FunctionInt2Int = ammonite.$sess.cmd3$Helper$$anon$1@6c037ac4
substractOneV: FunctionInt2Int = ammonite.$sess.cmd3$Helper$$anon$2@6ad40383

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [5]:
assert(call(addOneV, 5) == 6)

In [6]:
call(substractOneV, 5)

res5: Int = 4

### Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), ...:

In [7]:
trait Function1[A, B]{
    def apply(a: A): B
}

trait Function2[A, B, C]{
    def apply(a: A, b: B): C
}

// up to Function22

defined trait Function1
defined trait Function2

Using these standard classes, we can create the `addOneV` function-value in a similar way: 

In [8]:
val addOneV: Function1[Int, Int] = new Function1[Int, Int]{
    def apply(a: Int): Int = 
        a + 1
}

addOneV: Function1[Int, Int] = ammonite.$sess.cmd7$Helper$$anon$1@29802222

But we can do it more easily, since Scala also provides special syntax to declare function types and create functions (so-called _lambda expressions_):

In [9]:
val addOneV: Int => Int = 
    (a: Int) => a + 1

val substractOneV: Int => Int = 
    (a: Int) => a - 1

addOneV: Int => Int = ammonite.$sess.cmd8$Helper$$Lambda$2153/956358442@1b9a2499
substractOneV: Int => Int = ammonite.$sess.cmd8$Helper$$Lambda$2154/1318747662@5c917930

And we can also profit from type inference:

In [10]:
val addOneV: Int => Int = 
    a => a + 1

val substractOneV: Int => Int = 
    a => a - 1

addOneV: Int => Int = ammonite.$sess.cmd9$Helper$$Lambda$2162/1963587052@4d4539aa
substractOneV: Int => Int = ammonite.$sess.cmd9$Helper$$Lambda$2163/439225711@9d67dcf

Using these syntactic facilities we can write the `call` HOF more easily: 

In [11]:
def call(int2int: Int => Int, number: Int): Int = 
    // int2int.apply(number)
    int2int(number)

defined function call

which we can use as follows:

In [12]:
call(addOneV, 1)
call(substractOneV, 1)

res11_0: Int = 2
res11_1: Int = 0

We can even pass function-methods that are converted on the fly to function-values (this is the so-called _eta-expansion_):

In [13]:
call(addOneM, 1)
call(substractOneM, 1)

res12_0: Int = 2
res12_1: Int = 0

Finally, for functions of two arguments we can use a similar syntax as well. So, instead of writing the more verbose:

In [14]:
val sum: Function2[Int, Int, Int] = new Function2[Int, Int, Int]{
    def apply(a: Int, b: Int): Int = 
         a + b
}

sum: Function2[Int, Int, Int] = ammonite.$sess.cmd13$Helper$$anon$1@2a1590d5

we can create a lambda expressions for a `Function2` value in the following way:

In [15]:
val sum: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b

sum: (Int, Int) => Int = ammonite.$sess.cmd14$Helper$$Lambda$2187/1074631685@7e32ce32

or, exploiting type inference:

In [16]:
val sum: (Int, Int) => Int = 
    (a, b) => a + b 

sum: (Int, Int) => Int = ammonite.$sess.cmd15$Helper$$Lambda$2193/39053294@50ab9e37

### Improved unit testing

In [17]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

Using higher-order functions we can create test catalogues which are parameterised by the function to be tested. For instance, let's consider the following two alternative implementations:

In [18]:
// Recursively

def sumR(list: List[Int]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => head + sumR(tail) : Int 
    }

defined function sumR

In [19]:
// With tail-recursion

def sumTR(list: List[Int]): Int = {

    def sumAux(acc: Int, list: List[Int]): Int = 
        list match {
            case Nil => acc : Int
            case head :: tail => sumAux(head + acc, tail) : Int 
        }
    
    sumAux(0, list)
}

defined function sumTR

Instead of creating ad-hoc test catalogues for each alternative, we can create a single one that receives the function to be tested as argument as follows:

In [20]:
/*
object TestSumR extends FlatSpec with Matchers{
    "length" should "work" in {
        sumR(List()) shouldBe 0 
        sumR(List(1)) shouldBe 1 
        sumR(List(1,2,3,4)) shouldBe 10
    }
}
*/

class TestSum(sum: List[Int] => Int) extends FlatSpec with Matchers{
    "length" should "work" in {
        sum(List()) shouldBe 0 
        sum(List(1)) shouldBe 1 
        sum(List(1,2,3,4)) shouldBe 10
    }
}

defined class TestSum

Now, we can test the `sumR` and `sumTR` functions by reusing the same test catalogue:

In [21]:
run(new TestSum(sumR))

cmd19$Helper$TestSum:
length
- should work


In [22]:
run(new TestSum(sumTR))

cmd19$Helper$TestSum:
length
- should work


## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [23]:
def isEvenLength: String => Boolean = 
    (s: String) => s.length % 2 == 0

defined function isEvenLength

This function is somehow the combination of two more basic functions `length` and `isEven`:

In [24]:
def length: String => Int = 
    s => s.length

defined function length

In [25]:
def isEven: Int => Boolean = 
    i => i % 2 == 0

defined function isEven

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` from the functions `length` and `isEven`? We can use a HOF which is able to compose functions:

In [26]:
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    (a: A) => f2(f1(a))

defined function compose

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [27]:
val isEvenLength: String => Boolean = 
    compose(isEven, length)

isEvenLength: String => Boolean = ammonite.$sess.cmd25$Helper$$Lambda$2981/1052563935@1f1e626d

The HOF `compose` is actually defined by `Function1`: 

In [28]:
val isEvenLength: String => Boolean = 
    isEven.compose(length)

isEvenLength: String => Boolean = scala.Function1$$Lambda$2986/29648588@6dae8ac6

or using infix notation:

In [29]:
val isEvenLength: String => Boolean = 
    isEven compose length

isEvenLength: String => Boolean = scala.Function1$$Lambda$2986/29648588@67fadc91

Note that a similar function to `compose`, called `andThen`, is also available: 

In [30]:
val isEvenLength: String => Boolean = 
    length andThen isEven

isEvenLength: String => Boolean = scala.Function1$$Lambda$318/1443055846@1721ec49

## HOFs as modularity devices

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [31]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }

defined function sum

In [32]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => head * multiply(tail)
    }

defined function multiply

These functions clearly share a common logic; their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [33]:
def combine(list: List[Int])(nil: Int, cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, combine(tail)(nil, cons))
    }

defined function combine

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [34]:
def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)

defined function sum

In [35]:
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)

defined function multiply

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`'s):

In [36]:
def foldRight[A, B](list: List[A])(nil: B, cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil, cons))
    }

defined function foldRight

The implementation of `sum` and `multiply` using `foldRight` is no more difficult:

In [37]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(0, _ + _)

defined function sum

In [38]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a: Int, b: Int) => a * b)

defined function multiply

although you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [38]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

In [39]:
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }

defined function foldRight

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

In [40]:
foldRight(List(1,2,3))(1)(_*_)

res39: Int = 6

<span style="background-color:powderblue;font-size:300%;">    
Exercise
</span>

Write a function `and` which receives a lists of booleans and returns the conjunction of all of them: 

In [41]:
def and(list: List[Boolean]): Boolean = ???

defined function and

Write a function `concat` that receives a lists of strings and return their concatenation:

In [42]:
def concat(list: List[String]): String = ???

defined function concat

## The Hall of Fame of HOFs

Besides `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [43]:
def foldLeft[A, B](b: B, f: (B, A) => B): List[A] => B = ???
def filter[A](f: A => Boolean): List[A] => List[A] = ???
def map[A, B](f: A => B): List[A] => List[B] = ???
def flatMap[A, B](f: A => List[B]): List[A] => List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

In [44]:
List(1,2,3).foldLeft(List[Int]())((l, e) => e :: l)
List(1,2,3).foldRight(List[Int]())(_ :: _)


res43_0: List[Int] = List(3, 2, 1)
res43_1: List[Int] = List(1, 2, 3)